In [1]:
from DemoServerDB import ConnectDatabase
from mapping import doc

In [2]:
temp = ConnectDatabase()

In [3]:
temp.conn

<connection object at 0x7fde9284b910; dsn: 'dbname=postgresdemotwo user=clientdemo password=xxxxxxxxxxx host=postgresdemotwo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com port=5432 sslmode=require', closed: 0>

In [4]:
doc['column_map']['TRANSACTION_MASTER']

{'cust_id': 'userid',
 'product_id': 'product_id',
 'quantity': 'quantity',
 'renewal': 'renewal',
 'revenue': 'revenue',
 'timestamp': 'purchase_date'}

In [5]:
query = 'SELECT "%s","%s" FROM "%s"'%(doc['column_map']['TRANSACTION_MASTER']['cust_id'],
                               doc['column_map']['TRANSACTION_MASTER']['product_id'],
                               doc['table_map']['TRANSACTION_MASTER'])

In [6]:
print query

SELECT "userid","product_id" FROM "trans"


In [7]:
temp.conn.rollback()

In [8]:
temp.cur.execute(query)

ProgrammingError: column "userid" does not exist
LINE 1: SELECT "userid","product_id" FROM "trans"
               ^


In [9]:
col_names = [desc[0] for desc in temp.cur.description]

TypeError: 'NoneType' object is not iterable

In [ ]:
col_names

In [ ]:
import pandas as pd

In [ ]:
trans_table = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [ ]:
trans_table.head()

In [ ]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'}, inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'}, inplace=True)

In [ ]:
trans_table.head()

In [ ]:
trans_table.shape

In [ ]:
trans_table = trans_table[['cust_id','product_id']]

In [ ]:
len(trans_table['cust_id'].unique())

In [ ]:
grp_cust = trans_table.groupby('cust_id').count()

In [ ]:
grp_cust.head()

In [ ]:
grp_cust = grp_cust.reset_index()

In [ ]:
grp_cust.head()

In [ ]:
grp_cust.shape

In [ ]:
grp_cust.rename(columns={'product_id':'count'}, inplace=True)

In [ ]:
grp_cust.head()

In [ ]:
grp_cust_df_1 = grp_cust.query('count != 1')

In [ ]:
grp_cust_df_1.shape

In [ ]:
grp_cust.shape

In [ ]:
grp_cust_df_1.reset_index(drop=True, inplace=True)

In [ ]:
grp_cust_df_1.head()

In [ ]:
grp_cust_df_1.tail()

In [ ]:
cust = grp_cust_df_1[['cust_id']]

In [ ]:
cust.head()

In [ ]:
final_data = trans_table.merge(cust, on='cust_id')

In [ ]:
final_data.shape

In [ ]:
cust_prod_comb = final_data.groupby(['cust_id','product_id']).size().reset_index().rename(columns={0:'count'})

In [ ]:
cust_prod_comb.head()

In [ ]:
affinity_df = cust_prod_comb.merge(grp_cust_df_1, on='cust_id')

In [ ]:
affinity_df.head()

In [ ]:
affinity_df.rename(columns={'count_x':'combination'}, inplace=True)
affinity_df.rename(columns={'count_y':'visits'}, inplace=True)

In [ ]:
affinity_df.head()

In [ ]:
affinity_df['affinity'] = affinity_df['combination'] / affinity_df['visits'] * 100

In [ ]:
affinity_df.head()

In [ ]:
affinity_df['affinity_round'] = affinity_df.affinity.round()

In [ ]:
affinity_df.head()

In [ ]:
affinity_user_df = affinity_df[['cust_id','affinity_round']]

In [ ]:
affinity_user_df.head()

In [ ]:
affinity_user_df.shape

In [ ]:
affinity_user_df1 = affinity_user_df.drop_duplicates('cust_id').reset_index(drop = True)

In [ ]:
affinity_user_df1.shape

In [ ]:
affinity_user_comb = affinity_user_df1.groupby(['affinity_round','cust_id']).size().reset_index().rename(columns={0:'users_count'})

In [ ]:
affinity_user_comb.head()

In [ ]:
affinity_user_comb.tail()

In [ ]:
affinity_user_comb_df = affinity_user_comb.groupby(by=['affinity_round'])['users_count'].sum()

In [ ]:
affinity_user_comb_df.head()

In [ ]:
type(affinity_user_comb_df)

In [ ]:
affinity_user_comb_df = affinity_user_comb_df.to_frame()

In [ ]:
affinity_user_comb_df.head()

In [ ]:
affinity_user_comb_df.reset_index(inplace=True)

In [ ]:
affinity_user_comb_df

In [ ]:
affinity_user_comb_df.users_count.sum()

In [ ]:
affinity_user_comb_df.sort_index(ascending=False,inplace=True)

In [ ]:
affinity_user_comb_df.head()

In [ ]:
affinity_user_comb_df['user_per'] = affinity_user_comb_df['users_count']/affinity_user_comb_df['users_count'].sum() * 100

In [ ]:
affinity_user_comb_df.head()

In [ ]:
affinity_user_comb_df['aggregated_per'] = affinity_user_comb_df.user_per.cumsum()

In [ ]:
affinity_user_comb_df.head()

In [ ]:
affinity_user_comb_df

In [ ]:
affinity_user_comb_df['round_aggr'] = affinity_user_comb_df.aggregated_per.round()

In [ ]:
affinity_user_comb_df.head()

In [ ]:
affinity_user_comb_df.tail()

In [ ]:
affinity_user_comb_df

In [ ]:
a = affinity_user_comb_df.loc[affinity_user_comb_df['round_aggr'] <= 2]

In [ ]:
a

In [ ]:
a.reset_index(drop=True, inplace=True)

In [ ]:
a

In [ ]:
a_value = a.affinity_round.loc[a['round_aggr'] <= 2]

In [ ]:
a_value = a_value.tolist()

In [ ]:
a_value

In [ ]:
a_value[-1]

In [ ]:
min()

In [ ]:
b = affinity_user_comb_df.loc[(affinity_user_comb_df['round_aggr'] >2) & 
                                       (affinity_user_comb_df['round_aggr'] <=10)]

In [ ]:
b

In [ ]:
b.reset_index(drop=True, inplace=True)

In [ ]:
b

In [ ]:
b_value = b.affinity_round.loc[(b['round_aggr'] >2) & 
                          (b['round_aggr'] <=10)]

In [ ]:
b_value

In [ ]:
b_value = b_value.tolist()

In [ ]:
b_value

In [ ]:
c = affinity_user_comb_df.loc[(affinity_user_comb_df['round_aggr'] >10) & 
                                       (affinity_user_comb_df['round_aggr'] <=25)]

In [ ]:
c

In [ ]:
c.reset_index(drop=True, inplace=True)

In [ ]:
c_value = c.affinity_round.loc[(c['round_aggr'] >10) & 
                                       (c['round_aggr'] <=25)]

In [ ]:
c_value

In [ ]:
c_value = c_value.tolist()

In [ ]:
c_value

In [ ]:
d = affinity_user_comb_df.loc[(affinity_user_comb_df['round_aggr'] >25) & 
                                       (affinity_user_comb_df['round_aggr'] <=50)]

In [ ]:
d

In [ ]:
d.reset_index(drop=True, inplace=True)

In [ ]:
d_value = d.affinity_round.loc[(d['round_aggr'] > 25) & 
                          (d['round_aggr'] <=50)]

In [ ]:
d_value

In [ ]:
d_value= d_value.tolist()

In [ ]:
d_value

In [ ]:
e = affinity_user_comb_df.loc[affinity_user_comb_df['round_aggr'] > 50]

In [ ]:
e.reset_index(drop=True, inplace=True)

In [ ]:
e_value = e.affinity_round.loc[e['round_aggr'] > 50]

In [ ]:
e_value

In [ ]:
e_value = e_value.tolist()

In [ ]:
e_value

In [ ]:
affinity_df['Recommendation_class'] = ""

In [ ]:
affinity_df.head()

In [ ]:
affinity_df.loc[affinity_df['affinity_round'] >= a_value[-1], 'Recommendation_class'] = "Class_A"

In [ ]:
affinity_df.head()

In [ ]:
affinity_df.loc[(affinity_df['affinity_round'] >= b_value[-1]) & (affinity_df['affinity_round'] < a_value[-1]), 'Recommendation_class'] = "Class_B"

In [ ]:
affinity_df.loc[(affinity_df['affinity_round'] >= c_value[-1]) & (affinity_df['affinity_round'] < b_value[-1]), 'Recommendation_class'] = "Class_C"

In [ ]:
affinity_df.loc[(affinity_df['affinity_round'] >= d_value[-1]) & (affinity_df['affinity_round'] < c_value[-1]), 'Recommendation_class'] = "Class_D"

In [ ]:
affinity_df.loc[(affinity_df['affinity_round'] >= e_value[-1]) & (affinity_df['affinity_round'] < d_value[-1]), 'Recommendation_class'] = "Class_E"

In [ ]:
affinity_df.head()

In [ ]:
#affinity_df.query('affinity_round >= 13')

In [ ]:
affinity_df.shape

In [ ]:
len(affinity_df.cust_id.unique())